In [ ]:
import numpy as np 
import pandas as pd 
from  matplotlib import pyplot
import seaborn as sns

#Let's review the rates variance per number of covered per state.

For instance how different would be to get cover for gold level if you are alone vs with wife and 2 dependents in CA vs WA.

In [ ]:
# Retrieve Rates
rate_fields = ['StateCode','PlanId','IndividualRate','PrimarySubscriberAndThreeOrMoreDependents','BusinessYear','Age','RatingAreaId']
rate_chunks = pd.read_csv("../input/Rate.csv",iterator=True,chunksize = 1000,usecols=rate_fields)
rates = pd.concat(chunk for chunk in rate_chunks)

# Retrieve plan attributes
attr_fields = ['PlanId','MetalLevel','BusinessYear','StateCode']

attr_chunks = pd.read_csv("../input/PlanAttributes.csv",iterator=True,chunksize = 1000,usecols=attr_fields)
attributes = pd.concat(chunk for chunk in attr_chunks)
attributes['PlanId']=attributes['PlanId'].str[0:14] # removing unneccessary post-fix

attributes_sorted=attributes.sort_values('PlanId', ascending=True)
rates_sorted=rates.sort_values('PlanId', ascending=True)

## Rationale
Let's pretend that someone is shopping across multiple states for insurance. The person is interested in 2016 Low plans and is trying to decide wether he wants insurance just for himself and to cover his family as well. 

To support the case we need to filter the data:
 - leave only 2016 plans
 - leave only Low plans
 - leave only plans with Family options

In [ ]:
rates_filtered=rates_sorted[rates_sorted.BusinessYear==2016]
rates_filtered=rates_filtered[rates_filtered.Age=='Family Option']

attributes_filtered=attributes_sorted[attributes_sorted.BusinessYear==2016]
attributes_filtered=attributes_filtered[attributes_filtered.MetalLevel=='Low']

In [ ]:
aggregate_rate=rates_filtered.groupby(['PlanId']).agg({'IndividualRate': np.mean,
                                        'PrimarySubscriberAndThreeOrMoreDependents': np.mean}).reset_index()
aggregate_rate['IndVSFamilyRate']=aggregate_rate['IndividualRate']/aggregate_rate['PrimarySubscriberAndThreeOrMoreDependents']
aggregate_rate=aggregate_rate[~aggregate_rate.IndVSFamilyRate.isin([np.inf,1,0,np.nan])]

In [ ]:
merge_plan_rates = pd.merge(attributes, aggregate_rate, how='inner', on=['PlanId'])

In [ ]:
state_agg=merge_plan_rates.groupby('StateCode').agg({'IndividualRate': np.mean,
                                        'PrimarySubscriberAndThreeOrMoreDependents': np.mean,
                                                          'IndVSFamilyRate': np.mean}).reset_index()

plt = sns.barplot(state_agg.sort(['IndVSFamilyRate'], ascending=[0]).StateCode, state_agg.IndVSFamilyRate,palette="Blues")
sns.plt.show()

In [ ]:
g = sns.pairplot(state_agg, hue="StateCode")